In [34]:
import glob
import numpy as np
import os
import re
import pandas as pd
import os.path
import xml.etree.ElementTree as etree
import csv
import getopt
import sys
import json
import copy
import tensorflow.keras.utils as TKU
from random import shuffle
from sklearn import utils
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from gensim.models.keyedvectors import KeyedVectors
import keras
import keras.preprocessing.text as kpt
import nltk
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, LSTM, Input, Embedding
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, concatenate, Activation, Average
from keras.models import Model
from keras import optimizers
from keras.callbacks import CSVLogger
from nltk.tokenize import TreebankWordTokenizer
from keras.models import model_from_json

In [10]:
inputDir = 'C:\\Users\\morzm\\jup_txts\\csv'
outputDir = 'C:\\Users\\morzm\\jup_txts\\csv\\output'
modelDir = 'C:\\Users\\morzm\\jup_txts\\csv\\model'

In [11]:
xml_path = "C:\\Users\\morzm\\jup_txts\\csv\\en\\text\\"
label_path = "C:\\Users\\morzm\\jup_txts\\csv\\en\\truth.txt"
label_file = open(label_path)
model_path = os.path.join(modelDir, 'en_model.json')
dic_path = os.path.join(modelDir, 'en_dictionary.json')
twit_path = os.path.join(modelDir, 'twitter_en_data_set.csv')

In [12]:
# xml to cvs

with open(twit_path, 'w+', encoding='utf-8') as csvfile:
    fieldnames = ['text', 'gender']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for filename in os.listdir(xml_path):
        if not filename.endswith('.xml'): continue
        # ----reading file name
        find_file_name = os.path.splitext(filename)[0]
        # print(find_file_name)
        # finding author gender
        label_file = open(label_path)
        for row in label_file:
            if find_file_name in row:
                # print(find_file_name)
                # print(row.split(':::')[1])
                gender = row.split(':::')[1]

        # print(gender)

        # reading xml full path
        xml_fullname = os.path.join(xml_path, filename)
        tree = etree.parse(xml_fullname)
        root = tree.getroot()
        # reading xml file data
        for i in range(len(root[0])):
            writer.writerow({'text': str(root[0][i].text), 'gender': str(gender)})
csvfile.close()

In [16]:
# tweak this code
# define tokenize, stem


my_df = pd.read_csv(twit_path)
my_df['gender'] = my_df['gender'].map({'female\r\n': 0, 'male\r\n': 1})
my_df['text'] = my_df['text'].apply(lambda x: " ".join(stem(x) for x in x.split()))
stop = stopwords.words('english')
my_df['text'] = my_df['text'].apply(lambda x: " ".join(x for x in x.split(" ") if x not in stop))
my_word_stop = ['the', 'in', 'of', 'is', 'a', 'to', 'an', 'be']
my_df['text'] = my_df['text'].apply(lambda x: " ".join(x for x in x.split(" ") if x not in my_word_stop))
my_df['text'] = my_df['text'].apply(lambda x: " ".join(tokenize(x) for x in x.split(" ")))
# my_df['text'] = my_df['text'].apply(lambda x: " ".join(x for x in text_processor.pre_process_doc(x) if x not in stop))
freq = pd.Series(' '.join(my_df['text']).split()).value_counts()[:50]
freq = list(freq.index)
my_df['text'] = my_df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))